In [1]:
!pip install scanpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [2]:
import pandas as pd
import scanpy as sc
import os

In [3]:
# new loader
def load_simulated_cna_dataset(folder_path):
    # Load raw counts
    expr = pd.read_csv(os.path.join(folder_path, "counts.matrix"), sep="\t", index_col=0)
    adata = sc.AnnData(expr.T)  # transpose so cells are rows

    # Load gene metadata
    gene_order = pd.read_csv(os.path.join(folder_path, "gene_order_file.txt"), sep="\t")
    gene_order = gene_order.set_index("gene_name")
    adata.var = gene_order.loc[adata.var_names].copy()

    # Load cell metadata
    annot = pd.read_csv(os.path.join(folder_path, "annotation.txt"), sep="\t")
    annot = annot.set_index("cell_name")
    adata.obs = annot.loc[adata.obs_names].copy()

    return adata

In [4]:
import zipfile
import os

In [39]:
zip_path = "sim_mixed_cna_goldstandard.zip"  # name of uploaded file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(".")

In [40]:
# Now you can access the folder like this:
adata = load_simulated_cna_dataset("sim_mixed_cna_goldstandard")

In [41]:
print(adata)

AnnData object with n_obs × n_vars = 500 × 2000
    obs: 'cell_type', 'simulated_cnvs', 'n_genes', 'n_counts'
    var: 'chromosome', 'start', 'end'


In [42]:
adata.write("sim_mixed_cna_goldstandard.h5ad")


In [43]:
from google.colab import files
files.download("sim_mixed_cna_goldstandard.h5ad")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
print(adata)


AnnData object with n_obs × n_vars = 500 × 2000
    obs: 'cell_type', 'simulated_cnvs', 'n_genes', 'n_counts'
    var: 'chromosome', 'start', 'end'


combine all 3 datasets

In [16]:
adata_small = sc.read_h5ad("sim_small_gain_lowfreq.h5ad")
adata_medium = sc.read_h5ad("sim_medium_gain_midfreq.h5ad")
adata_large = sc.read_h5ad("sim_large_gain_highfreq.h5ad")

In [17]:
adata_small.obs["simulation_group"] = "small_gain_lowfreq"
adata_medium.obs["simulation_group"] = "medium_gain_midfreq"
adata_large.obs["simulation_group"] = "large_gain_highfreq"


In [18]:
adata_combined = adata_small.concatenate(
    adata_medium,
    adata_large,
    batch_key="source",
    batch_categories=["small", "medium", "large"],
    index_unique=None  # prevent appending _1, _2 to cell names
)


<ipython-input-18-2c2b0f684131>:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_combined = adata_small.concatenate(
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [22]:
# Combine into a single 'chromosome' column
adata_combined.var["chromosome"] = adata_combined.var["chromosome-small"].combine_first(
    adata_combined.var["chromosome-medium"]).combine_first(
    adata_combined.var["chromosome-large"])

# Drop the old ones
adata_combined.var.drop(columns=["chromosome-small", "chromosome-medium", "chromosome-large"], inplace=True)


<ipython-input-22-d015bef54c26>:2: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  adata_combined.var["chromosome"] = adata_combined.var["chromosome-small"].combine_first(
<ipython-input-22-d015bef54c26>:3: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  adata_combined.var["chromosome-medium"]).combine_first(


In [23]:
print(adata_combined)
print(adata_combined.obs["simulation_group"].value_counts())


AnnData object with n_obs × n_vars = 1500 × 2000
    obs: 'cell_type', 'simulated_cnvs', 'n_genes', 'n_counts', 'simulation_group', 'source'
    var: 'start', 'end', 'chromosome'
simulation_group
large_gain_highfreq    500
medium_gain_midfreq    500
small_gain_lowfreq     500
Name: count, dtype: int64


In [24]:
adata_combined.write("sim_combined_cnas.h5ad")


In [25]:
from google.colab import files
files.download("sim_combined_cnas.h5ad")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
print(adata.obs)

           cell_type simulated_cnvs  n_genes  n_counts
Cell1    medium_gain           gain     1986     10212
Cell2         normal           none     1984      9958
Cell3    medium_gain           gain     1986     10230
Cell4    medium_gain           gain     1992     10176
Cell5    medium_gain           gain     1990     10367
...              ...            ...      ...       ...
Cell496   large_gain           gain     1990     10845
Cell497  medium_gain           gain     1984     10252
Cell498  medium_gain           gain     1984     10172
Cell499   large_gain           gain     1984     10873
Cell500   small_gain           gain     1989     10063

[500 rows x 4 columns]
